In [113]:
import urllib.request
import bs4
import pandas

response = urllib.request.urlopen('https://vk.com/topic-22522055_25921627?offset=20')
soup = bs4.BeautifulSoup(response, 'html.parser')

In [114]:
nodes = soup.findAll('div', {'class': 'post_item'})

In [115]:
import datetime as _dt
import re as _re

monthRus = [
    'янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'
]

vkDateRegex = _re.compile('([0-9]{1,2})\W([а-я]{3})\W([0-9]{4})\Wв\W([0-9]{1,2}):([0-9]{1,2})', _re.IGNORECASE)
props = []

In [125]:
for node in nodes:
    fromId = node.find('a', {'class': 'pi_author'}).get('href')[1:]
    
    toId = node.find('a', {'class': 'mem_link'})
    toId = toId.get('href')[1:] if toId else 0
    
    date = node.find('a', {'class', 'item_date'}).text
    day, month, year, hours, minutes = vkDateRegex.findall(date)[0]
    date = _dt.datetime(int(year), monthRus.index(month)+1, int(day), int(hours), int(minutes), 0)
    
    likes = node.find('b', {'class', 'v_like'})
    likes = likes.text if likes else 0
    
    props.append({
        'from' : fromId,
        'to' : toId,
        'date' : date.strftime('%Y-%m-%d %H:%M:%S'),
        'likes' : likes
    })

In [126]:
separator = ',';
keys = props[0].keys()

In [127]:
from os import stat as _os_stat
from os import path as _os_path
if not _os_path.isfile('vk.csv') or _os_stat('vk.csv').st_size == 0:
    file = open('vk.csv', 'w')
    file.write(separator.join(keys)+'\n')
    file.close()

In [128]:
csv = open('vk.csv', 'a')
for prop in props:
    print(prop.values())

dict_values(['dimka_tlt', 'id40586669', '2012-01-30 21:25:00', '1'])
dict_values(['id100606467', 'sberbankhelp', '2012-01-30 21:28:00', '19'])
dict_values(['guglberg', '', '2012-01-30 23:22:00', '3'])
dict_values(['rumyantsev68', '', '2012-01-31 08:45:00', '5'])
dict_values(['sberbankhelp', 'alexeeev', '2012-01-31 10:06:00', '1'])
dict_values(['sberbankhelp', 'id146778911', '2012-01-31 10:23:00', '2'])
dict_values(['sberbankhelp', 'id100606467', '2012-01-31 10:40:00', 0])
dict_values(['sberbankhelp', 'dimka_tlt', '2012-01-31 10:46:00', 0])
dict_values(['sberbankhelp', 'guglberg', '2012-01-31 10:47:00', 0])
dict_values(['sberbankhelp', 'olya_smetanina', '2012-01-31 11:04:00', '3'])
dict_values(['olya_smetanina', '', '2012-01-31 11:25:00', '1'])
dict_values(['dimka_tlt', 'sberbankhelp', '2012-01-31 11:55:00', '3'])
dict_values(['sberbankhelp', 'rumyantsev68', '2012-01-31 12:36:00', 0])
dict_values(['id140995465', '', '2012-02-01 11:48:00', '3'])
dict_values(['sberbankhelp', 'id140995465'

In [93]:
props[0]['date'].strftime('%Y-%m-%d %H:%M:%S')

'2012-01-30 21:25:00'